In [1]:
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 7.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 69.4 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import PeftModel

# --- 1. Paths ---
base_model_path = "Salesforce/codet5-base"
adapter_path = "/kaggle/input/pyfix/tensorflow2/default/1" 
# --- 2. Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# --- 3. Load Base Model with 8-bit Quantization ---
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto"
)

# --- 4. Load and Merge LoRA Adapter ---
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.merge_and_unload()  # Merges LoRA weights for final inference
model.eval()

print("model is loaded and merged")


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:85: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


model is loaded and merged


In [4]:
import json

def load_and_preprocess_data(file_path):
    """Loads and preprocesses the CodeNet dataset for training."""
    
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)  # Load all data at once

    preprocessed_data = []

    for i, entry in enumerate(data):
        
            preprocessed_data.append(entry)

    return preprocessed_data

# Test with 10 samples
test_file = "/kaggle/input/code-net-python/test.jsonl"
test_data_sample = load_and_preprocess_data(test_file)

print(test_data_sample[0])




{'src_id': 'p02547_s853646132', 'src': ['N', '=', 'int', '(', 'input', '(', ')', ')', 'NEW_LINE', 'z', ',', 'w', '=', '[', ']', ',', '[', ']', 'NEW_LINE', 'K', '=', '0', 'NEW_LINE', 'for', 'i', 'in', 'range', '(', 'N', ')', ':', 'NEW_LINE', 'INDENT', 'x', ',', 'y', '=', 'map', '(', 'int', ',', 'input', '(', ')', '.', 'split', '(', ')', ')', 'NEW_LINE', 'z', '.', 'append', '(', 'x', ')', 'NEW_LINE', 'w', '.', 'append', '(', 'y', ')', 'NEW_LINE', 'DEDENT', 'for', 'j', 'in', 'range', '(', '1', ',', 'N', '-', '1', ')', ':', 'NEW_LINE', 'INDENT', 'if', 'z', '[', 'j', ']', '==', 'w', '[', 'j', ']', 'and', 'z', '[', 'j', '+', '1', ']', '==', 'w', '[', 'j', '+', '1', ']', 'and', 'z', '[', 'j', '+', '2', ']', '==', 'w', '[', 'j', '+', '2', ']', ':', 'NEW_LINE', 'INDENT', 'K', '+=', '1', 'NEW_LINE', 'DEDENT', 'DEDENT', 'if', 'K', '>=', '1', ':', 'NEW_LINE', 'INDENT', 'print', '(', '" Yes "', ')', 'NEW_LINE', 'DEDENT', 'else', ':', 'NEW_LINE', 'INDENT', 'print', '(', '" No "', ')', 'NEW_LINE', 'D

In [12]:
def run_inference_tokenized(model, tokenizer, sample, max_input_length=768, max_output_length=768):
    # 1. Join the input tokens into a string (just like in training)
    input_text = " ".join(sample["src"])

    # 2. Tokenize input text
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_length,
        padding="max_length"
    ).to(model.device)

    # 3. Generate output (using sampling for diverse outputs)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=max_output_length,
            do_sample=True,  # Enable sampling
            top_k=50,        # Use top-k sampling for more diversity
            top_p=0.95,      # Nucleus sampling (top_p)
            temperature=1.0, # Randomness in predictions
            no_repeat_ngram_size=2,  # Prevent repeating ngrams
        )

    # 4. Decode output tokens to text
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    return input_text, decoded_output


In [13]:
# Pick a test sample
sample = test_data_sample[0]

# Run inference
input_text, predicted_fix = run_inference_tokenized(model, tokenizer, sample)

# Print results
print("🔹 INPUT (flattened tokens):\n", input_text)
print("\n🔹 PREDICTED FIX:\n", predicted_fix)
print("\n🔹 GROUND TRUTH:\n", " ".join(sample["tgt"]))

🔹 INPUT (flattened tokens):
 N = int ( input ( ) ) NEW_LINE z , w = [ ] , [ ] NEW_LINE K = 0 NEW_LINE for i in range ( N ) : NEW_LINE INDENT x , y = map ( int , input ( ) . split ( ) ) NEW_LINE z . append ( x ) NEW_LINE w . append ( y ) NEW_LINE DEDENT for j in range ( 1 , N - 1 ) : NEW_LINE INDENT if z [ j ] == w [ j ] and z [ j + 1 ] == w [ j + 1 ] and z [ j + 2 ] == w [ j + 2 ] : NEW_LINE INDENT K += 1 NEW_LINE DEDENT DEDENT if K >= 1 : NEW_LINE INDENT print ( " Yes " ) NEW_LINE DEDENT else : NEW_LINE INDENT print ( " No " ) NEW_LINE DEDENT

🔹 PREDICTED FIX:
 N = int ( input ( ) ) NEW_LINE z , w = [ ] , [ [ 0 ] ] * N NEWLINE K = 0 NEW for i in range ( N ) : NEWLine INDENT x , y = map ( int , input [ i : i + 1 ] . split ( 2 ) . flatten ( 5 ) + [ int : * ( y - x ) ] ) - [ float : - 1 , - 2 : ] NEWS = list ( map [ x ] + y ) ** 2 NEWSH = ( w + 2 - j ) * 2 + 3 NEWT = k NEWINJ = - k / 2.0 NEWDEDENT if K >= 1 : return " Yes " NEW . format ( k ) [ : : 2 ] [ K ] ; NEWP INDENT if k [ j + N ] 